# Домашнее задание

В этом домашнем задании вам предлагается попрактиковаться в обучении моделей в Yandex DataSphere!

Вы будете решать задачу на части датасета Epsilon — это один из стандартных наборов данных, который часто используется для тестирования алгоритмов машинного обучения. Он был создан для задач регрессии и содержит сгенерированные данные, которые позволяют оценивать производительность моделей.

▎Описание датасета Epsilon:

• Цель: Задача регрессии, где цель состоит в предсказании целевой переменной на основе набора признаков.

• Признаки: Датасет содержит 2000 признаков, из которых 100 являются полезными для предсказания целевой переменной, а остальные 1900 — шумовыми (неинформативными).

• Целевая переменная: Целевая переменная представляет собой линейную комбинацию полезных признаков с добавлением случайного шума.

## Задание 1

Создайте новый проект в DataSphere.
Откройте Jupyter Notebook.
Запишите в первой ячейке строки ниже и запустите код.
При запуске выберите самую дешевую машину с 1 GPU (T4).

In [1]:
%pip install catboost -q


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from catboost import CatBoostRegressor
import pandas as pd
import timeit

## Задание 2

Скачайте данные для задачи из Yandex Object Storage по ссылке.

Далее выделите матрицу объект-признак X и вектор с целевой переменной y (это столбец 0 в исходных данных).

Сколько строк в датасете?

**Ответ:** 25000

In [3]:
Data = pd.read_csv("https://storage.yandexcloud.net/epsilon/epsilon.csv")
Data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,1.0,-0.005738,0.007481,0.015977,0.001949,0.035965,-0.012589,-0.005272,-0.018107,0.041178,-0.015026,-0.028819,-0.006537,0.033669,-0.034622,0.041462,-0.006984,0.015137,0.040556,-0.003575,-0.026854,-0.000848,-0.018508,0.037773,0.010988,0.001606,0.001236,-0.000411,0.001886,-0.016363,-0.007835,0.013054,-0.041248,-0.013555,0.029254,-0.022851,0.032329,0.015484,-0.024664,0.003483,...,0.004737,-0.013773,0.011465,0.001008,0.022264,0.008917,-0.016302,0.000937,0.013812,0.019966,-0.043475,-0.004776,0.009994,0.013874,0.039987,-0.030642,0.011992,-0.015720,0.008593,0.005424,0.028540,-0.037337,-0.000306,0.033081,0.035953,-0.018372,0.018434,-0.004643,0.006698,0.010878,-0.007748,-0.018985,0.017533,-0.016377,-0.025507,-0.007074,-0.039108,-0.027469,-0.008356,-0.030009
1,-1.0,-0.032287,0.016732,-0.017783,0.020227,-0.007429,0.038325,-0.013316,-0.004563,0.005065,0.027075,-0.004615,0.022719,-0.001174,-0.005253,0.017911,-0.008285,0.023018,-0.005886,-0.021725,0.008088,-0.001925,0.010360,0.008404,0.046580,-0.037502,-0.016718,-0.009010,0.018608,0.004694,0.008248,0.014946,-0.003008,-0.036849,0.026321,0.039021,0.029011,0.053509,0.008748,-0.026753,...,0.021166,0.033842,0.032127,0.009800,-0.004692,0.029451,0.003939,-0.024806,-0.023525,-0.009425,0.011778,0.000086,-0.013626,0.018617,0.002709,0.019048,0.007278,0.020253,-0.043801,0.055158,-0.021189,0.011909,-0.023126,-0.003718,0.002489,0.007376,0.020273,-0.040885,0.019845,-0.025208,-0.023597,-0.035897,-0.021217,0.001084,0.008088,0.032573,0.001252,0.003677,0.012997,0.009525
2,-1.0,-0.026042,0.005822,0.024423,-0.005320,0.010567,0.005121,0.019061,-0.005601,0.023020,0.001421,-0.014876,0.000457,0.018014,-0.026738,0.028460,-0.005551,0.032041,-0.001586,-0.001274,0.000041,-0.010229,-0.000169,0.026797,0.005379,0.013230,-0.026943,0.000167,-0.040692,0.001488,-0.004061,-0.009515,-0.024695,-0.009305,0.022785,-0.046229,-0.001362,0.021154,-0.004119,-0.007879,...,0.019149,-0.038718,-0.007682,-0.010387,-0.002312,-0.035631,-0.001417,0.005136,-0.022054,-0.024243,-0.023056,-0.005739,0.004376,0.026141,0.024937,-0.035690,0.010754,-0.010134,0.020673,-0.004343,-0.013395,-0.025811,-0.008312,0.035563,0.020299,-0.012911,-0.012708,0.013668,0.003074,-0.028061,-0.002573,-0.064170,0.003719,0.000383,-0.022022,-0.021736,-0.030644,-0.017614,0.008090,-0.023110
3,-1.0,-0.008883,0.021915,0.004105,-0.012184,0.037085,-0.026911,-0.001835,0.013376,0.039198,-0.010749,-0.038545,-0.002609,0.034121,-0.023056,0.039868,-0.020030,0.033576,0.035937,0.006193,-0.008194,0.007799,-0.018153,0.038572,0.003459,0.013427,0.009876,-0.009281,-0.009035,-0.004651,0.000801,0.012578,-0.039997,0.002461,0.026828,-0.014464,0.010976,-0.000261,-0.024141,0.006310,...,-0.028574,-0.008188,-0.003977,0.013828,0.018268,-0.007530,-0.014502,0.009532,-0.009569,0.027312,-0.030824,0.010323,0.022540,-0.005601,0.039152,-0.025755,0.020176,-0.015800,0.001929,-0.002211,0.018374,-0.033326,-0.005877,0.034189,0.038494,-0.034589,-0.002887,-0.007752,0.006511,0.016056,-0.007596,-0.017965,0.010094,0.003548,-0.004301,0.001076,-0.037762,-0.036552,-0.000607,-0.028872
4,-1.0,-0.011601,0.023786,-0.027328,-0.032168,-0.015888,-0.004879,0.083705,-0.057912,-0.023810,-0.019682,0.006109,0.001752,-0.025317,0.026342,-0.020087,0.033633,0.017105,-0.007755,-0.018530,0.005137,-0.031172,0.013526,-0.015305,0.011995,0.015383,-0.018211,0.015624,-0.000032,0.040280,0.016254,-0.068284,0.020131,-0.023008,-0.045610,0.022045,-0.019298,0.013363,-0.048987,0.038767,...,0.008275,0.036259,-0.019819,0.022745,-0.002223,-0.006338,-0.009873,0.024213,-0.002828,-0.021336,0.010702,0.001577,0.005220,-0.010850,-0.022982,0.003274,-0.002836,0.011834,-0.016946,0.014952,-0.006768,0.013806,0.018803,-0.003302,-0.022351,0.020366,0.006664,-

In [4]:
Data.shape

(25000, 2001)

In [5]:
X, y = Data.drop(columns=['0']), Data['0']

## Задание 3

При помощи train_test_split разбейте данные на тренировочную и тестовую часть в пропорции train : test = 3 : 1 и фиксируйте random_state = 42.

Сколько строк попало в test?

**Ответ:** 6250

In [6]:
# ваш код здесь
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [7]:
X_test.shape

(6250, 2000)

## Задание 4

Запустите функции train_on_cpu и train_on_gpu из урока на данных из задания и замерьте время выполнения. Во сколько раз быстрее модель обучается на GPU?

**Ответ:** 6

In [8]:
%%bash
mkdir -p /tmp/catboost_info
ls -l /tmp/catboost_info

total 0


In [9]:
def train_on_cpu():
    model = CatBoostRegressor(
    iterations=100,
    learning_rate=0.03,
    train_dir = '/tmp/catboost_info'
  )

    model.fit(
      X_train, y_train,
      eval_set=(X_test, y_test),
      verbose=10
  );

cpu_time = timeit.timeit('train_on_cpu()',
                         setup="from __main__ import train_on_cpu",
                         number=1)

print('Time to fit model on CPU: {} sec'.format(int(cpu_time)))

0:	learn: 0.9957802	test: 0.9958478	best: 0.9958478 (0)	total: 347ms	remaining: 34.4s
10:	learn: 0.9597630	test: 0.9614357	best: 0.9614357 (10)	total: 2.86s	remaining: 23.2s
20:	learn: 0.9326821	test: 0.9363627	best: 0.9363627 (20)	total: 5.31s	remaining: 20s
30:	learn: 0.9100316	test: 0.9151370	best: 0.9151370 (30)	total: 7.76s	remaining: 17.3s
40:	learn: 0.8920378	test: 0.8985657	best: 0.8985657 (40)	total: 10.2s	remaining: 14.7s
50:	learn: 0.8768287	test: 0.8849754	best: 0.8849754 (50)	total: 12.6s	remaining: 12.1s
60:	learn: 0.8628386	test: 0.8723240	best: 0.8723240 (60)	total: 15s	remaining: 9.61s
70:	learn: 0.8501568	test: 0.8608860	best: 0.8608860 (70)	total: 17.4s	remaining: 7.12s
80:	learn: 0.8392860	test: 0.8510374	best: 0.8510374 (80)	total: 19.8s	remaining: 4.66s
90:	learn: 0.8292436	test: 0.8421806	best: 0.8421806 (90)	total: 22.2s	remaining: 2.2s
99:	learn: 0.8206586	test: 0.8347525	best: 0.8347525 (99)	total: 24.3s	remaining: 0us

bestTest = 0.8347524696
bestIteration = 

In [10]:
def train_on_gpu():
    model = CatBoostRegressor(
    iterations=100,
    learning_rate=0.03,
    task_type='GPU',
    train_dir = '/tmp/catboost_info'
  )

    model.fit(
      X_train, y_train,
      eval_set=(X_test, y_test),
      verbose=10
  )

    return model


gpu_time = timeit.timeit('train_on_gpu()',
                         setup="from __main__ import train_on_gpu",
                         number=1)

print('Time to fit model on GPU: {} sec'.format(int(gpu_time)))

0:	learn: 0.9952701	test: 0.9956003	best: 0.9956003 (0)	total: 49ms	remaining: 4.85s
10:	learn: 0.9564941	test: 0.9581875	best: 0.9581875 (10)	total: 355ms	remaining: 2.88s
20:	learn: 0.9279700	test: 0.9320542	best: 0.9320542 (20)	total: 553ms	remaining: 2.08s
30:	learn: 0.9051331	test: 0.9109861	best: 0.9109861 (30)	total: 749ms	remaining: 1.67s
40:	learn: 0.8864570	test: 0.8946517	best: 0.8946517 (40)	total: 947ms	remaining: 1.36s
50:	learn: 0.8699146	test: 0.8796359	best: 0.8796359 (50)	total: 1.14s	remaining: 1.1s
60:	learn: 0.8553280	test: 0.8667777	best: 0.8667777 (60)	total: 1.34s	remaining: 858ms
70:	learn: 0.8423965	test: 0.8556583	best: 0.8556583 (70)	total: 1.54s	remaining: 629ms
80:	learn: 0.8311055	test: 0.8460624	best: 0.8460624 (80)	total: 1.74s	remaining: 407ms
90:	learn: 0.8204726	test: 0.8365693	best: 0.8365693 (90)	total: 1.93s	remaining: 191ms
99:	learn: 0.8119672	test: 0.8293228	best: 0.8293228 (99)	total: 2.11s	remaining: 0us
bestTest = 0.8293227659
bestIteration 

In [11]:
print(cpu_time/gpu_time)

6.046850524600179


## Задание 5

Модифицируйте функцию train_on_gpu() так, чтобы она возвращала обученную модель. С помощью обученной на тренировочных данных модели сделайте прогноз на тесте и вычислите значение метрики r2-score.

Округлите ответ до сотых. Какое число получилось?

**Ответ:** 0.31

In [12]:
# ваш код здесь
from sklearn.metrics import r2_score


model = train_on_gpu()
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R2-score: {r2:.2f}")


0:	learn: 0.9952701	test: 0.9956003	best: 0.9956003 (0)	total: 44.1ms	remaining: 4.37s
10:	learn: 0.9564941	test: 0.9581875	best: 0.9581875 (10)	total: 345ms	remaining: 2.79s
20:	learn: 0.9279700	test: 0.9320542	best: 0.9320542 (20)	total: 546ms	remaining: 2.05s
30:	learn: 0.9051331	test: 0.9109861	best: 0.9109861 (30)	total: 743ms	remaining: 1.65s
40:	learn: 0.8864570	test: 0.8946517	best: 0.8946517 (40)	total: 941ms	remaining: 1.35s
50:	learn: 0.8699146	test: 0.8796359	best: 0.8796359 (50)	total: 1.14s	remaining: 1.09s
60:	learn: 0.8553280	test: 0.8667777	best: 0.8667777 (60)	total: 1.33s	remaining: 854ms
70:	learn: 0.8423965	test: 0.8556583	best: 0.8556583 (70)	total: 1.53s	remaining: 627ms
80:	learn: 0.8311055	test: 0.8460624	best: 0.8460624 (80)	total: 1.73s	remaining: 406ms
90:	learn: 0.8204726	test: 0.8365693	best: 0.8365693 (90)	total: 1.93s	remaining: 191ms
99:	learn: 0.8119672	test: 0.8293228	best: 0.8293228 (99)	total: 2.1s	remaining: 0us
bestTest = 0.8293227659
bestIteratio

## Задание 6

При помощи GridSearchCV подберите гиперпараметры CatBoost на тренировочных данных:
* n_estimators: [10, 100, 200]
* max_depth: [4, 6, 8]

Примените обученную методом лучшую модель (best_estimator_) на тестовых данных и выведите r2-score. Округлите его до сотых. Чему он равен?

ВАЖНО! При подборе гиперпараметров может не хватать памяти, поэтому в этом случае в CatBoost поставьте `gpu_ram_part=0.5` - это будет означать, что вы ограничиваете использование памяти GPU 50-ю процентами (по умолчанию стоит 95%).

**Ответ:** 0.42 (но на Степике 0.41)

In [13]:
# ваш код здесь
from sklearn.model_selection import GridSearchCV


params = {
    'n_estimators': [10, 100, 200],
    'max_depth': [4, 6, 8]
}

grid_search = GridSearchCV(
    estimator=CatBoostRegressor(
        # iterations=100,
        learning_rate=0.03,  # если не указать, то R2 будет 0.51
        task_type='GPU',
        train_dir = '/tmp/catboost_info',
        gpu_ram_part=0.85,
        verbose=0
    ),
    param_grid=params,
    scoring='r2',
    cv=3,
    n_jobs=1,
    verbose=2
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R2-score: {r2:.2f}")

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .......................max_depth=4, n_estimators=10; total time=   2.0s
[CV] END .......................max_depth=4, n_estimators=10; total time=   1.8s
[CV] END .......................max_depth=4, n_estimators=10; total time=   1.8s
[CV] END ......................max_depth=4, n_estimators=100; total time=   2.4s
[CV] END ......................max_depth=4, n_estimators=100; total time=   2.5s
[CV] END ......................max_depth=4, n_estimators=100; total time=   2.4s
[CV] END ......................max_depth=4, n_estimators=200; total time=   3.1s
[CV] END ......................max_depth=4, n_estimators=200; total time=   3.3s
[CV] END ......................max_depth=4, n_estimators=200; total time=   3.1s
[CV] END .......................max_depth=6, n_estimators=10; total time=   1.8s
[CV] END .......................max_depth=6, n_estimators=10; total time=   1.9s
[CV] END .......................max_depth=6, n_es